In [23]:
# ???
# idea: multi-qubit gates formed by 2Q interactions, similar to multi-block
# maybe a V gate is defined by 2 points at iSwap, Delta by 3 points at iSwap
# gates can be defined by a vector of coordinates.
# only works because there is no 3Q interaction, just multiple 2Q interactions at the same time

In [24]:
# TODO
# given interaction sliders
# update colored points,
# update ouput final matrix

In [25]:
from abc import ABC
from ast import Pass
from hashlib import sha1

import numpy as np
import qutip

In [26]:
class QubitInteraction:
    def __init__(self, gc, gg, phi_c=0.0, phi_g=0.0):
        self.gc = gc
        self.gg = gg
        self.phi_c = phi_c
        self.phi_g = phi_g

    def construct_H(self, A, B):
        H_c = (
            np.exp(1j * self.phi_c) * A * B.dag()
            + np.exp(-1j * self.phi_c) * A.dag() * B
        )
        H_g = (
            np.exp(1j * self.phi_g) * A * B
            + np.exp(-1j * self.phi_g) * A.dag() * B.dag()
        )
        return self.gc * H_c + self.gg * H_g

In [27]:
class Hamiltonian(ABC):
    def _construct_U_lambda(self):
        return lambda t: (-1j * t * self.H).expm()


class QubitSystem(Hamiltonian):
    def __init__(self, qubit_interactions, num_qubits):
        self.H = 0
        self.num_qubits = num_qubits

        for qubit_interaction, qubit_pair in qubit_interactions:
            self.H += self.construct_interaction(qubit_interaction, qubit_pair)

    def construct_interaction(self, qubit_interaction, qubit_pair):
        a = qutip.operators.create(N=2)

        operators = [qutip.operators.identity(2) for _ in range(self.num_qubits)]
        operators[qubit_pair[0]] = a
        A = qutip.tensor(*operators)

        operators = [qutip.operators.identity(2) for _ in range(self.num_qubits)]
        operators[qubit_pair[1]] = a
        B = qutip.tensor(*operators)

        H_int = qubit_interaction.construct_H(A, B)
        return H_int

In [28]:
interaction = QubitInteraction(np.pi / 2, 0)
sys = QubitSystem([(interaction, (0, 1))], 2)

In [29]:
sys.H

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.         0.         0.         0.        ]
 [0.         0.         1.57079633 0.        ]
 [0.         1.57079633 0.         0.        ]
 [0.         0.         0.         0.        ]]

In [30]:
sys._construct_U_lambda()(t=1.0)

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

In [31]:
import networkx as nx
import matplotlib.pyplot as plt
from ipywidgets import interact


class ConversionGainPhaseHamiltonian(Hamiltonian):
    # ... Existing code ...

    def draw_graph(self, interactions):
        G = nx.Graph()

        for (q1, q2), _ in interactions.items():
            G.add_edge(q1, q2, weight=1)

        pos = nx.spring_layout(G)  # positions for all nodes
        nx.draw_networkx_nodes(G, pos, node_size=500)
        nx.draw_networkx_edges(G, pos)
        nx.draw_networkx_edge_labels(
            G,
            pos,
            edge_labels={
                (q1, q2): f"{w['gc']} / {w['gg']}"
                for (q1, q2), w in interactions.items()
            },
        )
        plt.show()

    def create_sliders(self, interactions):
        @interact
        def interact_with_parameters(**kwargs):
            for key, value in kwargs.items():
                qubit_pair, param = key.split("_")
                interactions[qubit_pair][param] = value

            self.draw_graph(interactions)
            self.update_hamiltonian(interactions)

    def update_hamiltonian(self, interactions):
        # Update self.H with the new interactions
        # Recompute the unitary and other dependent quantities
        pass

    def create_interactive_environment(self, interactions):
        sliders = {
            f"{qubit_pair}_{param}": (0, 10, 0.1)
            for qubit_pair, params in interactions.items()
            for param in params
        }
        self.create_sliders(sliders)

ModuleNotFoundError: No module named 'networkx'